# 02. Exploratory Data Analysis (EDA)

## Objective
- Phân tích hành vi tiêu thụ điện
- Phát hiện các mô hình thời vụ

## 1. Setup and Load Dataset

In [1]:
# Core (in requirements.txt)
import numpy as np
import pandas as pd
import sys
import os

# Visualization
import matplotlib.pyplot as plt

# Add the root directory to the path so Python can find the 'src' module
sys.path.append(os.path.abspath('../'))
from src.data_loader import load_raw_data

# Display configuration
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

# Check Python version (Requires 3.10.11 as specified in .python-version)
print(f"Python version: {sys.version}")

Python version: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


In [2]:
# Call the function from src/data_loader.py
try:
    df = load_raw_data()
    print("✅ Data loaded successfully!")
    print(f"📊 Shape: {df.shape[0]} rows, {df.shape[1]} customers.")
except Exception as e:
    print(f"❌ Error: {e}")

✅ Data loaded successfully!
📊 Shape: 140256 rows, 370 customers.


## 2. Time Series Visualization
- Chuỗi thời gian tổng
- Chuỗi theo consumer (mẫu đại diện)

## 3. Seasonality Analysis
### 3.1 Daily Seasonality
- Biểu đồ trung bình theo giờ

### 3.2 Weekly Seasonality
- Ngày trong tuần vs cuối tuần

### 3.3 Yearly Seasonality
- Biến động theo tháng / mùa

## 4. Load Distribution
- Histogram
- Boxplot

## 5. Statistical Summary
- Mean, Median, Std
- Min / Max

## 6. Key Insights
- Các mẫu thời vụ quan trọng
- Nhận dạng giờ cao điểm / thấp điểm

## 7. Conclusion
- Tóm tắt insight chính phục vụ modeling